# Adondevivir.com Web Scraping

I will be doing web scraping of apartment data from the website www.adondevivir.com using BeautifulSoup

In [ ]:
import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from selenium import webdriver
import time

# Creating file
filename = "apartments.csv"
f= open(filename, "w")
headers = "Address,Price,Apartments,Bedrooms,Area,Bathrooms,Parking Slots,Status,Delivery,Services,Date Published,Link\n"
f.write(headers)

# Searching for apartments
num_of_pages = 5
for page in range(1, num_of_pages):
    print("Page number ", page)
    browser = webdriver.Chrome("C:\chromedriver.exe")
    if (page == 1):
        page_search = "https://www.adondevivir.com/departamentos-en-venta-en-santiago-de-surco-a-estrenar.html"
    else:
        page_search = "https://www.adondevivir.com/departamentos-en-venta-en-santiago-de-surco-a-estrenar-pagina-" + str(page) + ".html"
    
    browser.get(page_search)
    page_html = browser.page_source
    page_soup = soup(page_html, "html.parser")
    page_results = page_soup.findAll("div", {"class":"postingCardInfo"})

    # Reading every page result 
    for result in page_results:
        
        # Getting address of building
        address_span = result.findAll("span", {"class":"postingCardLocationTitle"})
        address = address_span[0].text.strip('\n\r\tS/": ')

        # Getting Number of Apartments, Area, Bedrooms, Bathrooms, Parking Slots for the building
        data = result.findAll("li")

        apartments = 'None'
        bedrooms = 'None'
        area = 'None'
        bathrooms = "None"
        parking_slots = "None"

        for i in range(len(data)):
            line = data[i].findAll("i")
            if "iconUnits" in line[0]['class']:
                apartments = data[i].text.strip('tuni.\n\r\t": ')
            elif "iconBedrooms" in line[0]['class']:
                bedrooms = data[i].text.strip('dorm.\n\r\t": ')
            elif "iconArea" in line[0]['class']:
                area = data[i].text.strip('m².\n\r\t": ')
            elif "iconBathroom" in line[0]['class']:
                bathrooms = data[i].text
            elif "iconGarage" in line[0]['class']:
                parking_slots = data[i].text.replace("estac.", "").strip()

        # Go to link to get more detailed information about the building and each apartment in the building
        link_list = result.findAll("a", {"class":"go-to-posting"})
        new_link = "https://www.adondevivir.com" + linkList[0]['href'] 
        browser = webdriver.Chrome("C:\chromedriver.exe")
        browser.get(new_link)
        time.sleep(1)
        page_html_new = browser.page_source

        # Getting additional data of building
        page_soup_new = soup(page_html_new, "html.parser")
        data_container = page_soup_new.findAll("div", {"id":"reactGeneralFeatures"})
        if (len(data_container) > 0):
            data_container = data_container[0]
            info = data_container.findAll("li")
            if (info):
                services = info[0].text.strip()
                for i in range(1, len(info)):
                    services = services + '-' + info[i].text.strip()
            else:
                services = "None"

        # Getting building status
        data_container = page_soup_new.findAll("div", {"class":"current"})
        if (len(data_container) > 0):
            status = data_container[0].text.strip('\n\r\tS/",: ')
        else:
            status = "None"

        # Getting delivery status
        data_container = page_soup_new.findAll("div", {"class":"row"})
        if (len(data_container) > 0):    
            delivery = data_container[0].text.strip('\n\r\t",: ').replace("\n\t\t\t\t\n\n", " ")
        else:
            delivery = "None"

        # Getting number of parking slots
        if (parking_slots == "None"):
            if (len(data_container)> 5):
                parking_slots = data_container[5].text.strip('\n\r\t",: ').replace("\n\t\t\t\t\n\n", " ")

        # Getting date of publication

        data_container = page_soup_new.findAll("span", {"class":"section-date"})
        if (len(data_container) > 0):
            date_published= data_container[0].text.replace("Publicado hace", "").strip('\n\r\tS/",: ')
        else:
            date_published = "None"

        # Getting data of each apartment in the building

        data_container = page_soup_new.findAll("div", {"class": "AccordionItem-td4pyq-0"} )
        if (len(data_container) > 0):
            for i in range(len(data_container)):
                bedrooms = data_container[i].findAll("h4", {"class": "AccordionTitle-td4pyq-2"})[0].text
                bedrooms = bedrooms[17:18]
                new_data = data_container[i].findAll("div", {"class": "sc-bkzZxe"})
                for j in range(len(new_data)):
                    price_text = new_data[j].findAll("div", {"class": "sc-idOhPF" })
                    area_text = new_data[j].findAll("li")
                    price = price_text[0].text.strip('\n\r\tS/",: ').replace(",", "")
                    area = area_text[0].text
                    if (len(area_text) > 1):
                        bathrooms = area_text[1].text.strip('\n\r\tS/"bBaños,: ')
                    else:
                        bathrooms = "None"
        
            # Writing data for each apartment
            f.write(address.replace(",", "-") + "," + price + "," + apartments + "," + bedrooms + "," + area + "," + 
                bathrooms + "," + parking_slots.replace("Estacionamientos ", "") + "," + status + "," + 
                delivery.replace("Entrega ", "") + "," + services + "," + date_published + "," + new_link + "\n") 
                
f.close()